## **RAG의 핵심, 문서 검색기 Retriever**
### **사용자의 쿼리를 재해석하여 검색하다, MultiQueryRetriever**
**Chroma DB에 문서 벡터 저장**

In [ ]:
import os
from dotenv import load_dotenv

# 환경변수 읽어오기
load_dotenv(override=True)  # .env 파일을 덮어쓰기 모드로 읽기

# 환경변수 불러오기
openai_key = os.getenv("OPENAI_API_KEY")
anthropic_key = os.getenv("ANTHROPIC_API_KEY")
huggingface_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

print(f"openai key values ::: {openai_key}")  # 테스트용 (실제 서비스에서는 print 금지)
print(f"anthropic key values ::: {anthropic_key}")  # 테스트용 (실제 서비스에서는 print 금지)
print(f"huggingface_token::: {huggingface_token}")  # 테스트용 (실제 서비스에서는 print 금지)

In [ ]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
result = embedding_model.embed_query("테스트 문장입니다.")
print(result[:5])  #

Test 소스

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma

#헌법 PDF 파일 로드

# PDF파일 불러올 객체 PyPDFLoader 선언 - window11 
loader = PyPDFLoader(r"G:\내 드라이브\LLM-RAG-LangChain\대한민국헌법(헌법제1호).pdf")

# PDF파일 불러올 객체 PyPDFLoader 선언 - macOS
# loader = PyPDFLoader(
#     r"/Users/youngho_moon/Library/CloudStorage/GoogleDrive-anskong@gmail.com/내 드라이브/LLM-RAG-LangChain/대한민국헌법(헌법)(제00010호)(19880225).pdf"
# )
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# docs = text_splitter.split_documents(pages)

# Chroma를 제거하고 임베딩만 추출
docs = text_splitter.split_documents(pages[:3])  # 소량만 사용

# 임베딩 결과 확인
embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')
for doc in docs:
    vec = embedding_model.embed_query(doc.page_content)
    print(vec[:5])

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings                           
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma

import shutil

import logging

# 기본 로깅 설정
logging.basicConfig(level=logging.DEBUG)

# ChromaDB 관련 로거 활성화
logging.getLogger("chromadb").setLevel(logging.DEBUG)
logging.getLogger("chromadb.db").setLevel(logging.DEBUG)
logging.getLogger("chromadb.telemetry").setLevel(logging.INFO)

#헌법 PDF 파일 로드

# PDF파일 불러올 객체 PyPDFLoader 선언 - window11 
loader = PyPDFLoader(r"G:\내 드라이브\LLM-RAG-LangChain\대한민국헌법(헌법제1호).pdf")

# PDF파일 불러올 객체 PyPDFLoader 선언 - macOS
# loader = PyPDFLoader(
#     r"/Users/youngho_moon/Library/CloudStorage/GoogleDrive-anskong@gmail.com/내 드라이브/LLM-RAG-LangChain/대한민국헌법(헌법)(제00010호)(19880225).pdf"
# )
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages[:5])

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
# db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))
# db = Chroma.from_documents(
#     docs,
#     OpenAIEmbeddings(model="text-embedding-3-small"),
#     persist_directory="./chroma_db"  # 폴더 직접 지정
# )

shutil.rmtree("C:/temp/chroma_db", ignore_errors=True)

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')

# db = Chroma.from_documents(docs, embedding_model, persist_directory="C:/temp/chroma_db")


db = Chroma(
    embedding_function=embedding_model,
    persist_directory="C:/temp/chroma_db",
    collection_name="my_collection"
)

# 소규모 배치로 나누어 삽입
batch_size = 5
for i in range(0, len(docs), batch_size):
    batch = docs[i:i + batch_size]
    db.add_documents(batch)

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
DEBUG:chromadb.config:Starting component System
DEBUG:chromadb.config:Starting component Posthog
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'idempotency_key': 'stainless-python-retry-a64ae48b-9473-4daf-8622-4d9860383f44', 'post_parser': <function Embeddings.create.<locals>.parser at 0x000001BEDCDC7CE0>, 'json_data': {'input': [[27384, 165, 253, 241, 70821, 12554, 233, 162, 228, 110, 25333, 198, 16, 611, 220, 24, 8790, 113, 255, 62841, 28617, 243, 16306, 254, 30381, 42771, 30426, 25941, 169, 227, 250], [67945, 24486, 50273, 120, 89059, 255, 169, 245, 234, 28617, 243, 58, 30426, 169, 52375, 220, 6393, 23, 13, 22, 13, 1114, 13, 1483, 169, 245, 234, 28617, 243, 63171, 16, 48424, 11, 220, 6393, 23, 13, 22, 13, 1114, 2637, 63171, 30381, 60, 38187, 16, 41953, 3396, 112, 251, 14705, 243, 

: 

**질문을 여러 버전으로 재해석하여 Retriever에 활용**

In [ ]:
#```Chroma DB에 대한민국 헌법 PDF 임베딩 변환 및 저장하는 과정은 위 셀에 있습니다```
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

#질문 문장 question으로 저장
question = "국회의원의 의무는 무엇이 있나요?"
#여러 버전의 질문으로 변환하는 역할을 맡을 LLM 선언
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125",
                 temperature = 0)
#MultiQueryRetriever에 벡터DB 기반 Retriever와 LLM 선언
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

# 여러 버전의 문장 생성 결과를 확인하기 위한 로깅 과정
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

#여러 버전 질문 생성 결과와 유사 청크 검색 개수 출력
unique_docs = retriever_from_llm.invoke(input=question)
len(unique_docs)

In [ ]:
unique_docs